In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.colors import LinearSegmentedColormap

from chatbot_personalization.utils.helper_functions import get_base_dir_path

# Generate Figure 4

## Load data

In [ ]:
path = get_base_dir_path() / "data/ratings_and_matching.csv"

df = pd.read_csv(path)
df = pd.read_csv(path)
df = df.drop(columns=["utility", "user_id"])
df = df.set_index("assignments")

statements = df.columns

k = len(statements)

In [ ]:
group_to_label = {statements[i]: "G" + str(i + 1) for i in range(k)}
statement_to_label = {statements[i]: "S" + str(i + 1) for i in range(k)}
label_to_statement = {v: k for k, v in statement_to_label.items()}
df.rename(index=group_to_label, columns=statement_to_label, inplace=True)
df = df.sort_index(axis="columns")
df = df.sort_index(axis="index")


assignment_table = df.copy().astype(bool)
assignment_table[:] = False
for col in df.columns:
    assignment_table.loc["G" + col[1:], col] = True

levels = df.stack().astype(int)
matched_levels = df[assignment_table].stack().astype(int)

level_names = pd.Series(
    index=["not at all", "poorly", "somewhat", "mostly", "perfectly"],
    data=[0, 1, 2, 3, 4],
)

matched_level_frequencies = matched_levels.value_counts(normalize=True)
matched_level_frequencies = level_names.map(matched_level_frequencies).fillna(0)

level_frequencies = levels.value_counts(normalize=True)
level_frequencies = level_names.map(level_frequencies).fillna(0)

## Plotting code

In [ ]:
def make_pie_plot(pie_df, title):
    pie_df = pie_df.reset_index().copy()
    pie_df["legend_label"] = pie_df.apply(
        lambda row: f"{row['level']} ({100*row['frequency']:.0f}%)", axis=1
    )
    nonzero_pie_df = pie_df[pie_df["frequency"] > 0]
    filtered_frequencies = nonzero_pie_df["frequency"]
    labels = nonzero_pie_df["level"].values
    colors = nonzero_pie_df["color"].values

    if "annotated_pie_chart" in title:
        print(colors)

    fig, ax = plt.subplots(figsize=(12, 5), subplot_kw=dict(aspect="equal"))
    wedges, _ = ax.pie(
        filtered_frequencies, colors=colors, wedgeprops=dict(width=0.5), startangle=-40
    )

    kw = dict(arrowprops=dict(arrowstyle="-"), zorder=0, va="center")

    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1) / 2.0 + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = f"angle,angleA=0,angleB={ang}"
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        ax.annotate(
            labels[i],
            xy=(x, y),
            xytext=(1.4 * np.sign(x), 1.4 * y),
            horizontalalignment=horizontalalignment,
            fontsize=17,
            **kw,
        )

    from matplotlib.patches import Patch

    legend_handles = [
        Patch(facecolor=row["color"], label=row["legend_label"])
        for _, row in pie_df.iterrows()
    ]
    plt.legend(
        handles=legend_handles,
        loc="lower center",
        bbox_to_anchor=(0.5, -0.3),
        title=title,
        ncol=6,
        fontsize=12,
        title_fontsize=15,
    )

## Generate Figure 4

In [ ]:
color_map = LinearSegmentedColormap.from_list("", ["#8BA9EB", "#061B5F"])
base_color = "#3753A5"  # used for main paper

pie_df = pd.DataFrame(level_frequencies, columns=["frequency"])
pie_df.index.name = "level"
pie_df["value"] = pie_df.index.map(level_names)
pie_df["color"] = pie_df["value"].map(lambda v: color_map(v / pie_df["value"].max()))

In [ ]:
frequency = pd.DataFrame(matched_level_frequencies, columns=["frequency"])
pie_df["frequency"] = frequency
pie_df["color"] = ["#3753A5", "#5C71C8", "#A3B2FF", "#F1F1E6", "#C8D5FF"][::-1]
make_pie_plot(pie_df, title="How well does your assigned statement represent you?")
plt.savefig("fig4_assigned_utilities_pie_chart.pdf", bbox_inches="tight")
plt.show()